## Vapor Pressure Deficit exploration
### Matt and Dana Work Session 07 Aug 2023

In [ ]:
import pandas as pd
import math
import matplotlib.pyplot as plt

### FireFamilyPlus Dewpoint, Sat Vapor Pressure and VPD Calculations

In [ ]:
# Calculate Dewpoint temperature from Temp and RH
def CalcDPT(tempF, RH):
	safeTemp = min(140.0, tempF)
	safeRH = max(5.0, RH)
	dp = -398.36 - 7428.6 / (-15.674 + math.log(safeRH / 100.0 * exp(-7482.6/(safeTemp + 398.36) + 15.675)))
	return dp

# Calculate the Vapor Pressure from Temperature(F)
def  CalcVP(tempF):
    tmpC =  (tempF - 32.0) / 1.8
    vp = 610.7 * math.exp((17.38 * tmpC)/(239 + tmpC))
    #print(tmpC)
    return vp

# Calculate the VPD from RH and Temperature(F)
def CalcVPD(RH, TempF):
    vp = CalcVP(TempF)
    vpd = vp - (RH / 100) * vp
    if(vpd < 0.0):
        vpd = 0.0
    return vpd

# Calculate the Actual Vapor Pressure from VPSat(Pa) and RH(%)
def CalcVPAct(VPSat,RH):
    return (RH / 100) * VPSat


### First, read the FW21 into a Pandas DataFrame

In [ ]:
df = pd.read_csv("./Data/FW21//MSO.fw21.csv")

In [ ]:
df

### Let's fix some of the datatypes in the DataFrame

In [ ]:
siteName = "Missoula"
df.rename(columns=lambda x: x.strip(), inplace=True)
df['DateTime']=pd.to_datetime(df['DateTime'].astype(str))
df.set_index(df.DateTime,inplace=True)
df.DateTime.dtype

In [ ]:
# Filter for any bad data
df = df[(df['Precipitation(in)']<5)]

In [ ]:
df.columns

In [ ]:
df['Temperature(F)'].plot()

### Let's plot the Saturation Vapor Pressure curve

In [ ]:
f,ax =plt.subplots()
vp = []
for t in range(0,110):
    vp.append(CalcVP(t))
plt.plot(vp)
plt.grid()
ax.set_xlabel('Temperature (F)')
ax.set_ylabel('Vapor Pressure (Pa)')


### Let's calculate the Saturation Vapor Pressure for our weather observations

In [24]:
VPSat = df.apply(lambda row: CalcVP(row['Temperature(F)']),axis=1)
df = df.copy()
df['VPSat(Pa)'] = VPSat
#df.assign(df['VPSat(Pa)'] = df['Temperature(F)'])
#df['VPSat(Pa)'] = VPSat

In [ ]:
df.columns

df['VPSat(Pa)'].groupby(df['DateTime'].dt.dayofyear).mean(['VPSat(Pa)']).plot()

### Now, Let's calculate the Actual Vapor Pressure (VPAct)

In [ ]:
df.loc[:,'VPAct(Pa)'] = (df['RelativeHumidity(%)']/ 100) * df['VPSat(Pa)']

https://colorbrewer2.org/#type=diverging&scheme=RdYlBu&n=5

### Let's plot the differences between VPAct and VPSat

In [ ]:
f,ax = plt.subplots()
df['VPSat(Pa)'].groupby(df['DateTime'].dt.dayofyear).mean(['VPSat(Pa)']).plot(ax=ax,color='#d7191c',label='VPSat',legend=True)
df['VPAct(Pa)'].groupby(df['DateTime'].dt.dayofyear).mean(['VPAct(Pa)']).plot(ax=ax,color='#2c7bb6',label='VPAct',legend=True)
plt.grid()
ax.set_xlabel('Day of Year')
ax.set_ylabel('Vapor Pressure (Pa)')

In [ ]:
df.loc[:,'VPD(Pa)'] = df['VPSat(Pa)'] - df['VPAct(Pa)']

In [ ]:
varname = 'VPD(Pa)'
f,ax = plt.subplots()
df[varname].groupby(df['DateTime'].dt.dayofyear).min(varname).plot(ax=ax,color='#2c7bb6',label="Min",legend=True)
df[varname].groupby(df['DateTime'].dt.dayofyear).mean(varname).plot(ax=ax,color='#fee090',label="Mean",legend=True)
df[varname].groupby(df['DateTime'].dt.dayofyear).max(varname).plot(ax=ax,color='#d7191c',label="Max",legend=True)
plt.grid()
ax.set_title(f"Missoula {varname} (2001-2023)")
ax.set_xlabel("Day of year")
ax.set_ylabel(varname)

In [ ]:
varname = 'Precipitation(in)'
f,ax = plt.subplots()
df[varname].groupby(df['DateTime'].dt.dayofyear).min(varname).plot(ax=ax,color='#2c7bb6',label="Min",legend=True)
df[varname].groupby(df['DateTime'].dt.dayofyear).mean(varname).plot(ax=ax,color='#fee090',label="Mean",legend=True)
df[varname].groupby(df['DateTime'].dt.dayofyear).max(varname).plot(ax=ax,color='#d7191c',label="Max",legend=True)
plt.grid()
ax.set_title(f"Missoula {varname} (2001-2023)")
ax.set_xlabel("Day of year")
ax.set_ylabel(varname)

### Let's print some ranges to explore the data

In [ ]:
### Now let's plot them together (Sat and Act)

In [ ]:
print(df['WindSpeed(mph)'].min())
print(df['WindSpeed(mph)'].mean())
print(df['WindSpeed(mph)'].max())

In [ ]:
# Example of a data summary over a fixed time period
df['Temperature(F)'].resample('2Y').mean()

In [ ]:
df['Temperature(F)'].resample('3M').mean().plot(color='#2c7bb6')

### Let's plot annual summaries by percentiles

In [ ]:
varname = 'VPD(Pa)'
perc = 0.95
f,ax = plt.subplots()
df[varname].groupby(df['DateTime'].dt.year).quantile(perc).plot(ax=ax,color='#2c7bb6')
plt.grid()
ax.set_title(f"Annual {perc*100}th  percentile {varname} (2001-2023) (Missoula)")
ax.set_xlabel("Year")
ax.set_ylabel(varname)